In [16]:
!pip install --upgrade google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.0/98.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.0/299.0 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstalled tenacity-8.5.0
  Attempting uninstall: starlette
    Found existing installation: starlette 0.50.0
    Uninstalling starlette-0.50.0:
      Successfully uninstalled starlette-0.50.0
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.121.3
    Uninstalling fastapi-0.121.3:
      Successfully uninstalled fastapi-0.121.3
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.19.0
    Uninstalling google-cloud-storage-2.19.0:
      Successfully uninstalled google-cloud-s

In [98]:
!pip install -q google-adk[a2a]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.5/141.5 kB 3.7 MB/s eta 0:00:00


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x79606df497c0>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x79606ddf9a30>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x79606ddfbb30>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x79606de0bd70>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x79606c16ba40>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x796067f65190>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x796067f668a0>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x79606d8ca840>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x79606dd3e8a0>
ERROR:asyn

## Imports & Config

In [103]:
# -------------------------------
# Standard Library
# -------------------------------
import asyncio
import logging
import os
from typing import Any, Dict, List

# -------------------------------
# Google ADK – Agents
# -------------------------------
from google.adk.agents import Agent, LlmAgent, ParallelAgent, SequentialAgent, LoopAgent
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.callback_context import CallbackContext

# -------------------------------
# Google ADK – Models
# -------------------------------
from google.adk.models.google_llm import Gemini
from google.adk.models.llm_request import LlmRequest

# -------------------------------
# Google ADK – Tools
# -------------------------------
from google.adk.tools import AgentTool, FunctionTool, google_search, load_memory, preload_memory
from google.adk.tools.tool_context import ToolContext

# -------------------------------
# Google ADK – Runners
# -------------------------------
from google.adk.runners import InMemoryRunner, Runner

# -------------------------------
# Google ADK – Sessions & Memory
# -------------------------------
from google.adk.sessions import InMemorySessionService, DatabaseSessionService
from google.adk.memory import InMemoryMemoryService

# -------------------------------
# Google ADK – App / Plugins
# -------------------------------
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.plugins.base_plugin import BasePlugin
from google.adk.plugins.logging_plugin import LoggingPlugin

# -------------------------------
# Google ADK – A2A
# -------------------------------
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a

# -------------------------------
# Google GenAI Types
# -------------------------------
from google.genai import types



In [75]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

In [76]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

GEMINI_MODEL = "gemini-2.5-flash-lite"

## Sequential Agent Workflow


In [77]:
# Newsletter outline Agent
newsletter_outline_agent = Agent(
    name="Newsletter_Outline_Agent",
    model=Gemini(
        model=GEMINI_MODEL,
        retry_options=retry_config
    ),
    instruction="""You are an expert content strategist and newsletter editor.

Your task is to create a polished, well-structured newsletter outline for the provided topic.
Follow these requirements:

1. **Headline** — Write 1 compelling, curiosity-driven headline.
2. **Introduction** — Provide a short hook (2–3 sentences) that frames the topic and engages the reader.
3. **Main Sections** — Create 3–5 sections. For each section:
    - Give a concise section title.
    - Add 2–3 bullet points explaining the key ideas or angles to cover.
4. **Conclusion** — Provide a brief closing insight or call-to-action (1–2 sentences).
5. **Tone** — Keep the style clear, engaging, and suitable for a professional newsletter audience.
6. **Format** — Return the output in clean, structured markdown.

Only produce the outline. Do not include explanations of your reasoning.""",
    tools=[google_search],
    output_key="newsletter_outline",  # The result of this agent will be stored in the session state with this key.
)


In [78]:
# Newsletter initial writer Agent
newsletter_initial_writer_agent = Agent(
    name="Newsletter_InitialWriterAgent",
    model=Gemini(
        model=GEMINI_MODEL,
        retry_options=retry_config
    ),
    instruction="""You are a skilled narrative writer.

Your task is to generate the **first draft** of a short story based on the outline below.
Outline: {newsletter_outline}

Follow these rules:
- Length: **100–150 words**
- Style: Clear, engaging, and easy to read
- Structure: A beginning, middle, and end with a coherent narrative arc
- Voice: Show, don’t tell; avoid clichés
- Constraints:
    - Output **only** the story text
    - Do **not** include any prefaces, explanations, headings, or meta-commentary

Return only the story draft.""",
     tools=[google_search],
    output_key="newsletter_current_story",  # Stores the first draft in the state.
)

In [79]:
# Newsletter critic Agent
newsletter_critic_agent= Agent( name="Newsletter_CriticAgent",
    model=Gemini(
        model=GEMINI_MODEL,
        retry_options=retry_config
    ),
    instruction="""You are a precise and constructive story critic.

Review the story provided below and evaluate it based on:
- Plot coherence
- Character clarity and consistency
- Pacing and narrative flow
- Overall completeness (the story must feel whole and polished)

Story to review:
{newsletter_current_story}

Your rules:
1. If the story is **well-written, coherent, and feels complete**, you MUST respond with the exact phrase:
   **APPROVED**
2. Otherwise, provide **2–3 specific, detailed, and actionable suggestions** for improvement.
3. Do not include any summary, restatement of the story, or meta explanations—only the approval or the critique.

Return only the required output.""",
    output_key="newsletter_critique",  # Stores the feedback in the state.
)


def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

In [80]:
# Newsletter refiner Agent
newsletter_refiner_agent = Agent(
    name="Newsletter_RefinerAgent",
    model=Gemini(
        model=GEMINI_MODEL,
        retry_options=retry_config
    ),
    instruction="""You are a precise story refinement agent.

You will be given:
- The current story draft
- A critique from the critic agent

Story Draft:
{newsletter_current_story}

Critique:
{newsletter_critique}

Your task is:

1. **If the critique is EXACTLY the word "APPROVED":**
   - You MUST call the `exit_loop` function.
   - Do not output anything else.

2. **Otherwise:**
   - Rewrite the story draft to fully address and incorporate every point raised in the critique.
   - Keep the story between 100–150 words.
   - Preserve the original tone and narrative intent while improving clarity, coherence, and pacing.
   - Output only the revised story text with no explanations, meta-commentary, or formatting.

Follow these rules strictly.""",
    output_key="newsletter_current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

In [81]:
# # Email Agent (Maybe not agent)
# from google.genai.types import Content, Part

# user_input = f"Create a newsletter for category: {category}"
# new_message = Content(role="user", parts=[Part(text=user_input)])

## Runner Orchestration

In [82]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
newsletter_refinement_loop = LoopAgent(
    name="newsletter_RefinementLoop",
    sub_agents=[newsletter_critic_agent, newsletter_refiner_agent],
    max_iterations=2,
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="newsletter_Pipeline",
    sub_agents=[newsletter_outline_agent, newsletter_initial_writer_agent, newsletter_refinement_loop],
)


In [83]:
# runner = InMemoryRunner(agent=root_agent)
# response = await runner.run_debug(
#     "Write a newsletter on the most recent google developer blog posts"
# )

## Add Session

In [84]:
APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session

MODEL_NAME = "gemini-2.5-flash-lite"

In [85]:
# async def run_session(
#     runner_instance: Runner,
#     user_queries: list[str] | str = None,
#     session_name: str = "default",
# ):
#     print(f"\n ### Session: {session_name}")

#     # Get app name from the Runner
#     app_name = runner_instance.app_name

#     # Attempt to create a new session or retrieve an existing one
#     try:
#         session = await session_service.create_session(
#             app_name=app_name, user_id=USER_ID, session_id=session_name
#         )
#     except:
#         session = await session_service.get_session(
#             app_name=app_name, user_id=USER_ID, session_id=session_name
#         )

#     # Process queries if provided
#     if user_queries:
#         # Convert single query to list for uniform processing
#         if type(user_queries) == str:
#             user_queries = [user_queries]

#         # Process each query in the list sequentially
#         for query in user_queries:
#             print(f"\nUser > {query}")

#             # Convert the query string to the ADK Content format
#             query = types.Content(role="user", parts=[types.Part(text=query)])

#             # Stream the agent's response asynchronously
#             async for event in runner_instance.run_async(
#                 user_id=USER_ID, session_id=session.id, new_message=query
#             ):
#                 # Check if the event contains valid content
#                 if event.content and event.content.parts:
#                     # Filter out empty or "None" responses before printing
#                     if (
#                         event.content.parts[0].text != "None"
#                         and event.content.parts[0].text
#                     ):
#                         print(f"{MODEL_NAME} > ", event.content.parts[0].text)
#     else:
#         print("No queries!")

In [86]:
# session_service = InMemorySessionService()

# # Step 3: Create the Runner
# runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

# print("✅ Stateful agent initialized!")
# print(f"   - Application: {APP_NAME}")
# print(f"   - User: {USER_ID}")
# print(f"   - Using: {session_service.__class__.__name__}")

In [87]:
# await run_session(
#     runner,
#     [
#         "Write a newsletter on the most recent google developer blog posts"
#     ],
#     "stateful-agentic-session",
# )

## Add Memory

In [88]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")




✅ Helper functions defined.


In [91]:
APP_NAME = "Newsletter_app"
USER_ID = "demo_user"

memory_service = (
    InMemoryMemoryService()
)
session_service = InMemorySessionService()  # Handles conversations

# Create runner with BOTH services
runner = Runner(
    agent=root_agent,
    app_name="Newsletter_app",
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
)

In [92]:
# await run_session(
#     runner,
#     "Write a newsletter on the most recent google developer blog posts",
#     "conversation-01",  # Session ID
# )


### Session: conversation-01

User > Write a newsletter on the most recent google developer blog posts
Model: > # Unlocking the Future: Latest Google Developer Innovations

Google's developer ecosystem is buzzing with advancements, particularly in the realm of Artificial Intelligence and developer tools. Stay ahead of the curve with the latest insights and practical applications directly from the Google Developer Blog.

## Gemini 3: The Next Leap in Agentic AI

*   Explore the capabilities of Gemini 3 Pro, Google's most intelligent model yet, designed for advanced agentic coding and complex workflows.
*   Discover how new features like "thinking\_level" and "Thought Signatures" provide granular control over reasoning, latency, and context retention for sophisticated AI agents.
*   Learn about Google Antigravity, a new platform for building and orchestrating AI agents, and its integration with popular open-source frameworks.

## Enhancing Developer Productivity and User Experience

*  

## Add Observability

In [122]:
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str,
    session_id: str = "default"
):
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Normalize to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Run each query (just prints)
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            # Streaming debug prints only
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")

    # ✅ After pipeline finishes, fetch the TRUE FINAL OUTPUT
    final_state = await session_service.get_session(
        app_name=APP_NAME, user_id=USER_ID, session_id=session_id
    )

    output = final_state.state.get("newsletter_current_story")

    return output


In [104]:
# runner = InMemoryRunner(
#     agent=root_agent,
#     plugins=[
#         LoggingPlugin()
#     ],  # <---- 2. Add the plugin. Handles standard Observability logging across ALL agents
# )

In [105]:
# response = await runner.run_debug("Write a newsletter on the most recent google developer blog posts")

In [123]:
APP_NAME = "Newsletter_app"
USER_ID = "demo_user"

runner = Runner(
    agent=root_agent,
    app_name="Newsletter_app",
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
     plugins=[
        LoggingPlugin()
    ],
)

In [124]:
output = await run_session(
    runner,
    "Write a newsletter on the most recent google developer blog posts",
    "conversation-01",  # Session ID
)


### Session: conversation-01

User > Write a newsletter on the most recent google developer blog posts
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-0e545dbc-661f-43c8-b4e3-d9de81ae9b8e
[logging_plugin]    Session ID: conversation-01
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: Newsletter_app
[logging_plugin]    Root Agent: newsletter_Pipeline
[logging_plugin]    User Content: text: 'Write a newsletter on the most recent google developer blog posts'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-0e545dbc-661f-43c8-b4e3-d9de81ae9b8e
[logging_plugin]    Starting Agent: newsletter_Pipeline
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: newsletter_Pipeline
[logging_plugin]    Invocation ID: e-0e545dbc-661f-43c8-b4e3-d9de81ae9b8e
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: Newsletter_Outline_Agent
[logging_plugin]    Invocation ID: e-0e545dbc-661f-43c8-b4e3-d9de81ae

In [125]:
output

'Google\'s latest AI advancements empower developers to build smarter, more efficient applications. Gemini 3 Pro introduces advanced control over complex AI workflows with features like "thinking\\_level," while Google Antigravity simplifies agent creation and integration. This leads to smarter Google Home devices and enhanced Android app performance, including improved ANR monitoring.\n\nStreamline your AI development using updated Google AI Studio and Vertex AI tools. Explore cutting-edge research in real-time translation and generative UI. Harness the power of on-device AI with the Gemini 3 Pro Image model and Coral NPU platform for edge computing. Visit the Google Developer Blog for tutorials and case studies demonstrating the real-world impact of these tools, and start building the future today.'

## Add A2A

In [126]:
# Expose Agent as A2A
newsletter_a2a_app = to_a2a(
    root_agent, port=8001  # Port where this agent will be served
)

/tmp/ipython-input-1114529018.py:2: UserWarning: [EXPERIMENTAL] to_a2a: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  newsletter_a2a_app = to_a2a(
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/utils/agent_to_a2a.py:137: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  agent_executor = A2aAgentExecutor(
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/executor/a2a_agent_executor.py:101: Use

## Send out Emails

In [ ]:
# Only Pseudo code shown for proprietary purposes
email_list = []

def send_newsletter_email(recipient_email, recipient_name, newsletter_text):
    """Send a personalized newsletter email."""

    # Create email
    msg = MIMEMultipart()
    msg["From"] = EMAIL_ADDRESS
    msg["To"] = recipient_email
    msg["Subject"] = "Your Personalized Newsletter Story"

    # Customize the content
    email_body = f"""
Hi {recipient_name},

Here is the latest on Google Blog's this week:

{newsletter}

Have a great day!
"""

    msg.attach(MIMEText(email_body, "plain"))

    # Send it
    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.send_message(msg)

    print(f"Email sent to {recipient_name} <{recipient_email}>")